In [ ]:
#|default_exp distributed.models.spark.xgb

# SparkXGBForecast

> spark XGBoost forecaster

Wrapper of `xgboost.spark.SparkXGBRegressor` that adds an `extract_local_model` method to get a local version of the trained model and broadcast it to the workers.

In [ ]:
#|export
import xgboost as xgb
try:
    from xgboost.spark import SparkXGBRegressor  # type: ignore
except ModuleNotFoundError:
    import os
    
    if os.getenv('IN_TEST', '0') == '1':
        SparkXGBRegressor = object
    else:
        raise

In [ ]:
#|export
class SparkXGBForecast(SparkXGBRegressor):   
    def _pre_fit(self, target_col):
        self.setParams(label_col=target_col)
        return self

    def extract_local_model(self, trained_model):
        model_str = trained_model.get_booster().save_raw('ubj')
        local_model = xgb.XGBRegressor()
        local_model.load_model(model_str)
        return local_model